In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=123)

import os

### To obtain the predictions:
1. Replace the location of the test file and the output_path
2. Run the two commands to train our approach for validity and novelty (During training, the predictions for the test file would be generated after every epoch)
3. Extract the predictions as liklihood of novelty and validity

In [6]:
output_path = "../../data-ceph/arguana/argmining22-sharedtask/models/multitask" # Replace this with your equivelant path in your docker image: /mnt/ceph/storage/data-in-progress/data-research/arguana/argmining22-sharedtask/models/multitask

### Prepare the data:

In [8]:
taska_training_df = pd.read_csv('../data/TaskA_train.csv')
taska_valid_df    = pd.read_csv('../data/TaskA_dev.csv')
taska_test_df     = pd.read_csv('../data/TaskA_test.csv') #Replace this with the path to the test file

taska_training_df.insert(loc=0,column='row_num',value=np.arange(len(taska_training_df)))
taska_valid_df.insert(loc=0,column='row_num',value=np.arange(len(taska_valid_df)))
taska_test_df.insert(loc=0,column='row_num',value=np.arange(len(taska_test_df)))

#The </s></s>  is the separator used in the pre-trained nli model..
taska_training_df['input_txt'] = taska_training_df.apply(lambda x: '{}:{}  </s></s> {} '.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)
taska_valid_df['input_txt']    = taska_valid_df.apply(lambda x: '{}:{} </s></s> {}'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)
taska_test_df['input_txt']     = taska_test_df.apply(lambda x: '{}:{} </s></s> {}'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)

taska_validity_train_df = taska_training_df[taska_training_df.Validity != 0].copy()
taska_validity_valid_df = taska_valid_df[taska_valid_df.Validity != 0].copy()
taska_validity_test_df  = taska_test_df[taska_test_df.Validity != 0].copy()

taska_validity_train_df['label'] = taska_validity_train_df.Validity.apply(lambda x : "valid" if x == 1 else "invalid")
taska_validity_valid_df['label'] = taska_validity_valid_df.Validity.apply(lambda x : "valid" if x == 1 else "invalid")
taska_validity_test_df['label']  = taska_validity_test_df.Validity.apply(lambda x  : "valid" if x == 1 else "invalid")


taska_novelty_train_df = taska_training_df[taska_training_df.Novelty != 0].copy()
taska_novelty_valid_df = taska_valid_df[taska_valid_df.Novelty != 0].copy()
taska_novelty_test_df  = taska_test_df[taska_test_df.Novelty != 0].copy()

#Balancing the data for novelty task..
taska_novelty_train_balanced_df, y = ros.fit_resample(taska_novelty_train_df, taska_novelty_train_df['Novelty'])
taska_novelty_train_balanced_df['Novelty'] = y

taska_novelty_train_df['label'] = taska_novelty_train_df.Novelty.apply(lambda x : "novel" if x == 1 else "conservative")
taska_novelty_train_balanced_df['label'] = taska_novelty_train_balanced_df.Novelty.apply(lambda x : "novel" if x == 1 else "conservative")
taska_novelty_valid_df['label'] = taska_novelty_valid_df.Novelty.apply(lambda x : "novel" if x == 1 else "conservative")
taska_novelty_test_df['label']  = taska_novelty_test_df.Novelty.apply(lambda x  : "novel" if x == 1 else "conservative")

In [9]:
taska_validity_train_df[['row_num',  'label', 'input_txt']].to_csv('../data/multitask_data/validity_training_df.tsv', sep='\t', header=False, index=False)
taska_validity_valid_df[['row_num', 'label', 'input_txt' ]].to_csv('../data/multitask_data/validity_valid_df.tsv', sep='\t', header=False, index=False)
taska_validity_test_df[['row_num', 'label', 'input_txt' ]].to_csv('../data/multitask_data/validity_test_df.tsv', sep='\t', header=False, index=False)

taska_novelty_train_df[['row_num', 'label', 'input_txt']].to_csv('../data/multitask_data/novelty_training_df.tsv', sep='\t', header=False, index=False)
taska_novelty_train_balanced_df[['row_num', 'label', 'input_txt']].to_csv('../data/multitask_data/novelty_training_balanced_df.tsv', sep='\t', header=False, index=False)
taska_novelty_valid_df[['row_num', 'label', 'input_txt']].to_csv('../data/multitask_data/novelty_valid_df.tsv', sep='\t', header=False, index=False)
taska_novelty_test_df[['row_num', 'label', 'input_txt']].to_csv('../data/multitask_data/novelty_test_df.tsv', sep='\t', header=False, index=False)

### Train our Approach:
- We train one multitask model for novelty and one for validity with different task weights and learning rates.

In [10]:
! sh ./src-py/run_nli_based_multitask_experiment.sh ../data/multitask_data/nli_based_multitask_for_novelty.yml \
                                          ../data/multitask_data/ \
                                          ../../data-ceph/arguana/argmining22-sharedtask/models/multitask/nli_model/final-multitask-for-novelty/ \
                                          /var/argmining-sharedtask/nli-final-multitask-for-novelty\
                                          roberta-large-mnli_prepared_data 2e-5 10

Preparing data...
task object created from task file...
loading file https://huggingface.co/roberta-large-mnli/resolve/main/vocab.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/64a1d72b2bd05b0aff1a4dd9e7a90a6eea0312b4f914e80b0a923aa8f72219bd.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-large-mnli/resolve/main/merges.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/425529714b758f50b6d3f93f8093d859856fd41cf1cec7c8edf2ab44aee632b6.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-large-mnli/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/tokenizer_config.json from cache at None
loading configurati

In [11]:
! sh ./src-py/run_nli_based_multitask_experiment.sh ../data/multitask_data/nli_based_multitask_for_validity.yml \
                                          ../data/multitask_data/ \
                                          ../../data-ceph/arguana/argmining22-sharedtask/models/multitask/nli_model/final-multitask-for-validity/ \
                                          /var/argmining-sharedtask/nli-final-multitask-for-validity\
                                          roberta-large-mnli_prepared_data 5e-6 10

Preparing data...
task object created from task file...
loading file https://huggingface.co/roberta-large-mnli/resolve/main/vocab.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/64a1d72b2bd05b0aff1a4dd9e7a90a6eea0312b4f914e80b0a923aa8f72219bd.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-large-mnli/resolve/main/merges.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/425529714b758f50b6d3f93f8093d859856fd41cf1cec7c8edf2ab44aee632b6.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-large-mnli/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/tokenizer_config.json from cache at None
loading configurati

### Extract Predictions:

Look at the best epoch from the f1-score of the corrsponding task and choose the corrsponding generated predictions for the test.
- For novelty: The best f1-score on validation is 0.669 after the third epoch
- For validity: The best f1-score on validation is 0.73 after the third epoch

In [12]:
import sys
sys.path.append('./src-py')

from utils import *
from sklearn.metrics import precision_recall_fscore_support

In [13]:
best_novelty_pred = '../../data-ceph/arguana/argmining22-sharedtask/models/multitask/nli_model/final-multitask-for-novelty/NoveltyTask_test_predictions_3.tsv'
best_validity_pred = '../../data-ceph/arguana/argmining22-sharedtask/models/multitask/nli_model/final-multitask-for-validity/ValidityTask_test_predictions_4.tsv'

In [14]:
novelty_predictions_df = pd.read_csv(best_novelty_pred, delimiter='\t')
validity_predictions_df = pd.read_csv(best_validity_pred, delimiter='\t')

#prediction dictionary
novelty_preds  = pd.Series(novelty_predictions_df.score.values, index=novelty_predictions_df.uid).to_dict()
validity_preds = pd.Series(validity_predictions_df.score.values, index=validity_predictions_df.uid).to_dict()

In [15]:
taska_test_df = taska_test_df[(taska_test_df.Novelty!=0) & (taska_test_df.Validity!=0)]

In [16]:
taska_test_df['is_validity'] = taska_test_df.row_num.apply(lambda x: validity_preds[x])
taska_test_df['is_novelty']  = taska_test_df.row_num.apply(lambda x: novelty_preds[x])


taska_test_df['predicted validity'] = taska_test_df.is_validity.apply(lambda x: 1 if x >= 0.5 else -1)
taska_test_df['predicted novelty']  = taska_test_df.is_novelty.apply(lambda x: 1 if x >= 0.5 else -1)

In [17]:
taska_test_df.head()

,row_num,topic,Premise,Conclusion,Validity,Validity-Confidence,Novelty,Novelty-Confidence,Topic-in-dev-split,input_txt,is_validity,is_novelty,predicted validity,predicted novelty
0,0,Veal,Another selling point is that it's fast to coo...,"Veal appeals to young, fast-cooking",1,very confident,1,majority,no,Veal:Another selling point is that it's fast t...,0.815236,0.453287,1,-1
1,1,Veal,In addition to the unnecessary cruelty that mi...,Animal food is very expensive,-1,very confident,1,majority,no,Veal:In addition to the unnecessary cruelty th...,0.229442,0.179301,-1,-1
2,2,Veal,In addition to the unnecessary cruelty that mi...,Veal production is inefficient.,1,very confident,1,very confident,no,Veal:In addition to the unnecessary cruelty th...,0.193275,0.179444,-1,-1
3,3,Veal,In addition to the unnecessary cruelty that mi...,Veal meat production requires a huge amount of...,1,very confident,1,majority,no,Veal:In addition to the unnecessary cruelty th...,0.811837,0.271869,1,-1
4,4,Veal,In addition to the unnecessary cruelty that mi...,Veal production has a high carbon footprint.,-1,majority,-1,majority,no,Veal:In addition to the unnecessary cruelty th...,0.573908,0.179137,1,-1


Now the `taska_test_df` dataframe contains the predictions for validity and novelty... Submit it please :)

#### Compute scores on Test Set:

In [27]:
val_nov_metric(np.array(taska_test_df.is_validity.tolist()), np.array(taska_test_df.Validity.tolist()),
               np.array(taska_test_df.is_novelty.tolist()), np.array(taska_test_df.Novelty.tolist()))

{'f1_validity': 0.8656716417910448,
 'f1_novelty': 0.558139534883721,
 'f1_valid_novel': 0.46376811594202894,
 'f1_valid_nonnovel': 0.7638190954773869,
 'f1_nonvalid_novel': 0.3333333333333333,
 'f1_nonvalid_nonnovel': 0.5294117647058824,
 'f1_macro': 0.5225830773646579}

In [28]:
precision_recall_fscore_support(taska_test_df.Validity.tolist(), taska_test_df['predicted validity'], average='macro')

(0.8237762237762237, 0.7790684931506849, 0.7922108208955224, None)

In [29]:
precision_recall_fscore_support(taska_test_df.Novelty.tolist(), taska_test_df['predicted novelty'], average='macro')

(0.7306608426095533, 0.6720984020185029, 0.6723281944081526, None)

In [18]:
taska_test_df[['row_num', 'topic', 'Premise', 'Conclusion', 'predicted validity', 'predicted novelty']].to_csv('../data/output/final_predictions.csv')

#### Compute baseline scores:

In [5]:
singl_novelty_pred = "../../data-ceph/arguana/argmining22-sharedtask/models/multitask/nli_model/novelty/NoveltyTask_test_predictions_8.tsv"
single_validity_pred  = "../../data-ceph/arguana/argmining22-sharedtask/models/multitask/nli_model/novelty/NoveltyTask_test_predictions_8.tsv"

In [6]:
novelty_predictions_df = pd.read_csv(singl_novelty_pred, delimiter='\t')
validity_predictions_df = pd.read_csv(single_validity_pred, delimiter='\t')

#prediction dictionary
novelty_preds  = pd.Series(novelty_predictions_df.score.values, index=novelty_predictions_df.uid).to_dict()
validity_preds = pd.Series(validity_predictions_df.score.values, index=validity_predictions_df.uid).to_dict()

In [9]:
taska_test_df = taska_test_df[(taska_test_df.Novelty!=0) & (taska_test_df.Validity!=0)]

In [10]:
taska_test_df['nli_single_is_validity'] = taska_test_df.row_num.apply(lambda x: validity_preds[x])
taska_test_df['nli_single_is_novelty']  = taska_test_df.row_num.apply(lambda x: novelty_preds[x])

In [13]:
val_nov_metric(np.array(taska_test_df.nli_single_is_validity.tolist()), np.array(taska_test_df.Validity.tolist()),
               np.array(taska_test_df.nli_single_is_novelty.tolist()), np.array(taska_test_df.Novelty.tolist()))

{'f1_validity': 0.7467362924281984,
 'f1_novelty': 0.5457227138643068,
 'f1_valid_novel': 0.3780068728522337,
 'f1_valid_nonnovel': 0.0,
 'f1_nonvalid_novel': 0.0,
 'f1_nonvalid_nonnovel': 0.21348314606741572,
 'f1_macro': 0.14787250472991237}